In [0]:
%pip install databricks-langchain unitycatalog-langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.4/718.4 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 96.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 121.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 93.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 955.5/955.5 kB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 117.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 641.0/641.0 kB 30.1 

In [0]:
dbutils.library.restartPython()

In [0]:

%sql
-- I created this function to data from Samples.tpch tables . Calculates the total profit margin for a given market segment and year
CREATE OR REPLACE FUNCTION vectorcatalog.etl.get_profit_margin(
  market_segment STRING,
  order_year INT
)
RETURNS DOUBLE
COMMENT 'Calculates the total profit margin.'
RETURN (
  SELECT
    SUM(
      l.l_extendedprice * (1 - l.l_discount) - (ps.ps_supplycost * l.l_quantity)
    )
  FROM
    samples.tpch.lineitem AS l
  JOIN
    samples.tpch.orders AS o ON l.l_orderkey = o.o_orderkey
  JOIN
    samples.tpch.customer AS c ON o.o_custkey = c.c_custkey
  JOIN
    samples.tpch.partsupp AS ps
      ON l.l_partkey = ps.ps_partkey AND l.l_suppkey = ps.ps_suppkey
  WHERE
    c.c_mktsegment = market_segment
    AND YEAR(o.o_orderdate) = order_year
);

In [0]:
# Validate above function 

segment = 'BUILDING'
year = 1995

query = f"""
    SELECT 
        vectorcatalog.etl.get_profit_margin('{segment}', {year}) AS total_profit_margin
"""

# Execute the query
result_df = spark.sql(query)

# Display the result
display(result_df)

total_profit_margin
2.14471922674296E10


In [0]:
%sql

select * from  vectorcatalog.etl.segmentby_order_aggregated 

c_mktsegment,total_orders
HOUSEHOLD,1490949
AUTOMOBILE,1497982
BUILDING,1505153
MACHINERY,1499152
FURNITURE,1506764


In [0]:

%sql
--Creating new function to get total number of orders in the segment by_order_aggregated
CREATE or replace FUNCTION vectorcatalog.etl.get_total_orders()
RETURNS BIGINT
COMMENT 'Returns the total number of all orders in the segment by_order_aggregated.'
RETURN (
    SELECT COUNT(*) FROM vectorcatalog.etl.segmentby_order_aggregated
);


In [0]:
# Validate above function 

query = f"""
    SELECT 
        vectorcatalog.etl.get_total_orders() AS a_total_orders
"""

# Execute the query
result_df = spark.sql(query)

# Display the result
display(result_df)

a_total_orders
5


In [0]:
# Wrap the UC Function as a LangChain Tool
from databricks_langchain import UCFunctionToolkit

# Define the fully qualified names of the functions you want to load
function_list = [
    "vectorcatalog.etl.get_profit_margin",
    "vectorcatalog.etl.get_total_orders" 
]

# Create a toolkit with the list of function names
toolkit = UCFunctionToolkit(function_names=function_list)
tools = toolkit.tools

# Output to verify
print(f"Number of tools loaded: {len(tools)}")
print(f"Tool names for agent:")
for tool in tools:
    print(f"- {tool.name}")

Number of tools loaded: 2
Tool names for agent:
- vectorcatalog__etl__get_profit_margin
- vectorcatalog__etl__get_total_orders


In [0]:
# --- 0. Tool Initialization (Assuming get_total_orders and get_profit_margin are registered) ---
# Create and Run the Agent. Now, we define the LangChain agent using a Databricks-served LLM and the UC tool.
from databricks_langchain import ChatDatabricks
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from databricks_langchain.uc_ai import DatabricksFunctionClient, UCFunctionToolkit
import mlflow

# Initialize the Unity Catalog Function Client (automatically uses notebook/CLI credentials)
uc_client = DatabricksFunctionClient() 

# Define the function names to expose as tools
# NOTE: Make sure these names exactly match your registered Unity Catalog functions
TOOL_PROFIT_MARGIN = "vectorcatalog.etl.get_profit_margin"
TOOL_TOTAL_ORDERS = "vectorcatalog.etl.get_total_orders"

# The tools object contains the function metadata needed by the LLM
tools = UCFunctionToolkit(
    function_names=[TOOL_PROFIT_MARGIN, TOOL_TOTAL_ORDERS], 
    client=uc_client
).tools


# --- 1. Initialize the LLM client ---
LLM_ENDPOINT_NAME = "databricks-meta-llama-3-3-70b-instruct" 
llm = ChatDatabricks(endpoint=LLM_ENDPOINT_NAME, temperature=0.1)

# --- 2. Define the Agent Prompt (UPDATED) ---
system_prompt = (
    "You are a Senior Financial Analyst and a Data Query Engine. Your goal is to answer questions about "
    "profitability and order counts accurately. \n\n"
    "RULES:\n"
    "1. For any question related to **profit, margin, or financial performance**, you MUST use the "
    "`get_profit_margin` tool. This tool requires two parameters: a **market_segment (STRING)**, "
    "such as 'AUTOMOBILE', and an **order_year (INT)**, such as 1997.\n"
    "2. If the user asks to compare profitability across years or segments, you must perform multiple tool calls.\n"
    "3. For simple questions about the **total number of orders**, use the `get_total_orders` tool.\n"
    "4. If required parameters are missing for a tool call, you must first ask the user to clarify.\n"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# --- 3. Create the Agent ---
mlflow.langchain.autolog() 
agent = create_tool_calling_agent(llm, tools, prompt)

# --- 4. Create the Agent Executor ---
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# ----------------------------------------------------------------------
# --- 5. Invoke the Agent (Example runs demonstrating both tools) ------
# ----------------------------------------------------------------------

print("\n--- Running Agent (Tool-Calling: Profit Margin) ---")
# This question requires calling the complex function
result_profit = agent_executor.invoke({"input": "What was the profit margin for the 'BUILDING' segment in 1995?"})
print(f"Agent Final Output: {result_profit['output']}")

print("\n--- Running Agent (Tool-Calling: Total Orders) ---")
# This question requires calling the simple function
result_orders = agent_executor.invoke({"input": "Tell me the total count of all orders."})
print(f"Agent Final Output: {result_orders['output']}")



/databricks/python/lib/python3.12/site-packages/databricks/connect/session.py:475: UserWarning: Ignoring the default notebook Spark session and creating a new Spark Connect session. To use the default notebook Spark session, use DatabricksSession.builder.getOrCreate() with no additional parameters.
  warnings.warn(new_notebook_session_msg)



--- Running Agent (Tool-Calling: Profit Margin) ---


> Entering new AgentExecutor chain...


/local_disk0/.ephemeral_nfs/envs/pythonEnv-b0f42e7b-a050-4744-a99e-eda8007998ae/lib/python3.12/site-packages/unitycatalog/ai/core/databricks.py:600: UserWarning: The following parameters do not have descriptions: market_segment, order_year for the function vectorcatalog.etl.get_profit_margin. Using Unity Catalog functions that do not have parameter descriptions limits the functionality for an LLM to understand how to call your function. To improve tool calling accuracy, provide verbose parameter descriptions that fully explain what the expected usage of the function arguments are.
  check_function_info(function_info)
{"ts": "2025-09-30 21:07:15.776", "level": "ERROR", "logger": "pyspark.sql.connect.client.logging", "msg": "GRPC Error received", "context": {}, "exception": {"class": "_MultiThreadedRendezvous", "msg": "<_MultiThreadedRendezvous of RPC that terminated with:\n\tstatus = StatusCode.FAILED_PRECONDITION\n\tdetails = \"BAD_REQUEST: session_id is no longer usable. Generate a ne


Invoking: `vectorcatalog__etl__get_profit_margin` with `{'market_segment': 'BUILDING', 'order_year': 1995}`




/databricks/python/lib/python3.12/site-packages/databricks/connect/session.py:475: UserWarning: Ignoring the default notebook Spark session and creating a new Spark Connect session. To use the default notebook Spark session, use DatabricksSession.builder.getOrCreate() with no additional parameters.
  warnings.warn(new_notebook_session_msg)


{"format": "SCALAR", "value": "21447192267.4296"}The profit margin for the 'BUILDING' segment in 1995 was 21447192267.4296.

> Finished chain.
Agent Final Output: The profit margin for the 'BUILDING' segment in 1995 was 21447192267.4296.

--- Running Agent (Tool-Calling: Total Orders) ---


> Entering new AgentExecutor chain...

Invoking: `vectorcatalog__etl__get_total_orders` with `{}`


{"format": "SCALAR", "value": "5"}The total count of all orders is 5.

> Finished chain.
Agent Final Output: The total count of all orders is 5.


[Trace(trace_id=tr-562e04bed0ebc7e04c82917d135760a6), Trace(trace_id=tr-53b5c61405a132a2589e8e38ce4b0dac)]